In [1]:
import pandas as pd
import sys,os
from tqdm import tqdm
sys.path.insert(0, '/home/shenwanxiang/Research/aliyun_sync/COMPASS/')
from baseline.immnue_score.scorer import ssGSEA
from compass.utils import plot_embed_with_label
from compass import PreTrainer, FineTuner, loadcompass #, get_minmal_epoch
from compass.utils import plot_embed_with_label,plot_performance, score2
from compass.tokenizer import CANCER_CODE, CONCEPT

/home/shenwanxiang/anaconda3/envs/compass/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '/home/shenwanxiang/Research/aliyun_sync/COMPASS/paper/00_data/'
df_tpm = pd.read_pickle(os.path.join(data_path,  'TCGA.TPM.TABLE'))
df_label = pd.read_pickle(os.path.join(data_path, 'TCGA.PATIENT.PROCESSED.TABLE'))
df_label_org = pd.read_pickle(os.path.join(data_path, 'TCGA.PATIENT.TABLE'))

dfcx = df_label.cancer_type.apply(lambda x:x.replace('TCGA-', '')).map(CANCER_CODE).to_frame('cancer_code').join(df_tpm)

In [11]:
df_label_org.to_csv('./data/TCGA_clinical.csv')

In [4]:
from joblib import Parallel, delayed
def run_one(item):
    name, g_list = item
    gsea = ssGSEA(g_list, name)              # your original class
    score_df = gsea.fit_transform(df_tpm)    # rows=samples, one column: NES_{name}
    s = score_df.iloc[:, 0].copy()           # convert to a Series
    s.name = name                            # final column name = geneset name
    return name, s

In [5]:
gmt29  = pd.read_excel('./Fges_29_gmt.xlsx',sheet_name= 'Gene Description',skiprows=1)
gmt29 = gmt29.groupby(['Gene signature']).Gene.apply(lambda x:x.tolist())

items = list(gmt29.items())
results = Parallel(n_jobs=4)(delayed(run_one)(it) for it in tqdm(items))
dfc = pd.concat([s for _, s in results], axis=1)
dfc.loc[df_label.index].to_csv('./data/TCGA_ssGSEA_Fegs29.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [09:34<00:00, 19.81s/it]


In [8]:
def _connect(x):
    from itertools import chain
    return list(set(chain(*[i.split(':') for i in x])))
    
concept_genes = CONCEPT.groupby('BroadCelltypePathway').Genes.apply(lambda x:x.tolist()).apply(_connect)

items = list(concept_genes.items())
results = Parallel(n_jobs=4)(delayed(run_one)(it) for it in tqdm(items))
dfc = pd.concat([s for _, s in results], axis=1)
dfc.columns = dfc.columns.map(lambda x:x.replace('NES_', ''))
dfc = dfc[CONCEPT.groupby('BroadCelltypePathway').Concept_index.mean().sort_values().index]
dfc.loc[df_label.index].to_csv('./data/TCGA_ssGSEA_concept43.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 43/43 [14:11<00:00, 19.81s/it]


In [9]:
pretrainer = loadcompass('../../../checkpoint/latest/pretrainer.pt', map_location = 'cpu')
dfg, dfc = pretrainer.extract(dfcx,  batch_size= 128)

100%|###################################################################################| 80/80 [04:31<00:00,  3.39s/it]


In [10]:
dfc.to_csv('./data/TCGA_COMPASS_concept44.csv')